In [1]:
import sys
import os
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import json
import yaml
import pandas as pd
import mask_generator.settings as settings

pd.set_option('display.max_columns', None)

In [2]:
EXPERIMENTS_DIR = "../experiments"

def load_yaml(path: str) -> dict:
    with open(path, "r") as f:
        return yaml.safe_load(f)

def load_json(path: str) -> dict:
    with open(path, "r") as f:
        return json.load(f)

def load_run(run_path: str) -> dict:
    config_path = os.path.join(run_path, settings.config_filename)
    results_path = os.path.join(run_path, settings.results_filename)

    if not os.path.exists(config_path) or not os.path.exists(results_path):
        return {}

    config = load_yaml(config_path)
    results = load_json(results_path)

    return {
        "config": config,
        "results": results
    }

def flatten_dict(d: dict, parent_key: str = '', sep: str = '.') -> dict:
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def load_experiments(experiments_dir: str = EXPERIMENTS_DIR) -> pd.DataFrame:
    run_dirs = [os.path.join(experiments_dir, d) for d in os.listdir(experiments_dir)
                if os.path.isdir(os.path.join(experiments_dir, d))]

    data = []
    for run_dir in run_dirs:
        run_data = load_run(run_dir)
        if not run_data:
            continue
        flat_config = flatten_dict(run_data["config"])
        flat_results = flatten_dict(run_data["results"])
        summary = {**flat_config, **flat_results}
        summary["run_dir"] = run_dir
        data.append(summary)

    df = pd.DataFrame(data)
    if df.empty:
        raise ValueError("No valid experiment data found.")
    df.reset_index(drop=True, inplace=True)
    return df

In [3]:
if not os.path.exists(EXPERIMENTS_DIR):
    raise FileNotFoundError(f"Experiments directory '{EXPERIMENTS_DIR}' does not exist.")
df = load_experiments(EXPERIMENTS_DIR)
print(f"df columns: {df.columns.tolist()}")

df.to_csv(os.path.join(EXPERIMENTS_DIR, "experiments_summary.csv"), index=False)

df columns: ['model.arch', 'model.in_channels', 'model.out_channels', 'model.model_args.n_convs', 'model.model_args.filters', 'model.model_args.dropout', 'training.train_dataset', 'training.eval_dataset', 'training.seed', 'training.batch_size', 'training.num_epochs', 'training.lr', 'training.step_size', 'training.gamma', 'training.patience', 'training.delta', 'training.image_size', 'training.use_amp', 'training.qat.enabled', 'training.qat.backend', 'training.loss', 'other.name', 'other.run_hash', 'other.run_dir', 'other.git_commit', 'other.verbose', 'best_epoch', 'elapsed_time_sec', 'elapsed_time_str', 'test_metrics.loss', 'test_metrics.dice', 'test_metrics.iou', 'test_metrics.acc', 'test_metrics.precision', 'test_metrics.recall', 'test_metrics.f1', 'run_dir', 'model.model_args.encoder_name', 'model.model_args.encoder_weights', 'model.model_args.decoder_attention_type', 'model.model_args.activation', 'training.train_dataset_path', 'training.eval_dataset_path', 'training.train_image_siz

In [4]:
df1 = df.copy()

# Sort by test_metrics.dice
df1.sort_values(by=["test_metrics.dice"], ascending=False, inplace=True)
# Reset index after sorting
df1.reset_index(drop=True, inplace=True)
# Display the first 10 rows of the sorted DataFrame
print("Top 10 experiments sorted by test_metrics.dice:")
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(df1.head(10))

Top 10 experiments sorted by test_metrics.dice:


,model.arch,model.in_channels,model.out_channels,model.model_args.n_convs,model.model_args.filters,model.model_args.dropout,training.train_dataset,training.eval_dataset,training.seed,training.batch_size,training.num_epochs,training.lr,training.step_size,training.gamma,training.patience,training.delta,training.image_size,training.use_amp,training.qat.enabled,training.qat.backend,training.loss,other.name,other.run_hash,other.run_dir,other.git_commit,other.verbose,best_epoch,elapsed_time_sec,elapsed_time_str,test_metrics.loss,test_metrics.dice,test_metrics.iou,test_metrics.acc,test_metrics.precision,test_metrics.recall,test_metrics.f1,run_dir,model.model_args.encoder_name,model.model_args.encoder_weights,model.model_args.decoder_attention_type,model.model_args.activation,training.train_dataset_path,training.eval_dataset_path,training.train_image_size,training.augmentations
0,my_unet,3,1,2.0,"[32, 64, 128, 256]",0.0,"[{'csv': '/root/MaskGenerator/Datasets/simu_v0/simu.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}, {'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_val_target.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}]",[{'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_test.csv'}],42,32,100,0.001,10,0.1,30,0.0,"[128, 384]",False,False,fbgemm,"[{'name': 'lovasz_hinge', 'weight': 1.0, 'params': {'per_image': True}}]",20250630_161641_7a0e75ce26,7a0e75ce26,/root/MaskGenerator/experiments/20250630_161641_7a0e75ce26,2eb3422d4d580076e20677f6246425c1176b086e,False,69,8172.838023,02:16:12,0.406721,0.886859,0.794653,0.990496,0.884265,0.886896,0.885579,../experiments/20250630_161641_7a0e75ce26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,my_unet,3,1,2.0,"[32, 64, 128, 256]",0.0,"[{'csv': '/root/MaskGenerator/Datasets/simu_v0/simu.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}, {'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_val_target.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}]",[{'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_test.csv'}],42,32,100,0.001,10,0.1,30,0.0,"[128, 384]",False,False,fbgemm,"[{'name': 'lovasz_hinge', 'weight': 0.6, 'params': {'per_image': True}}, {'name': 'dice', 'weight': 0.4, 'params': {'smooth': 1.0}}]",20250630_204439_0a98e7389a,0a98e7389a,/root/MaskGenerator/experiments/20250630_204439_0a98e7389a,2eb3422d4d580076e20677f6246425c1176b086e,False,75,8178.645716,02:16:18,0.369370,0.883850,0.789748,0.990248,0.881743,0.883307,0.882524,../experiments/20250630_204439_0a98e7389a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,my_unet,3,1,2.0,"[32, 64, 128, 256]",0.0,"[{'csv': '/root/MaskGenerator/Datasets/simu_v0/simu.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}, {'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_val_target.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}]",[{'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_test.csv'}],42,32,100,0.001,10,0.1,30,0.0,"[128, 384]",False,False,fbgemm,"[{'name': 'dice', 'weight': 1.0, 'params': {'smooth': 1.0}}]",20250627_091953_3886f64417,3886f64417,/root/MaskGenerator/experiments/20250627_091953_3886f64417,4f87c2577adf0067632d7fe6a7148e1625bd3222,False,76,7963.275126,02:12:43,0.123984,0.877613,0.780423,0.989797,0.878868,0.874486,0.876671,../experiments/20250627_091953_3886f64417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,my_unet,3,1,2.0,"[32, 64, 128, 256]",0.0,"[{'csv': '/root/MaskGenerator/Datasets/simu_v0/simu.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}, {'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_val_target.csv', 'augmentations': ['geometry', 'drop